In [ ]:
import numpy as np
import math
from scipy.integrate import odeint
import matplotlib.pyplot as plt
%matplotlib inline 

Здесь мы импортировали 

In [ ]:
class Model:

    kp = np.array([60, 75, 30, 20])
    ki = np.array([0,0,0,0])
    x0 = np.array([math.pi,1*math.pi/180,0,0])
    xd = np.array([math.pi,0,0,0])
    dt = 0.1
    tt = np.array([0, dt])
    T = 10
    TT = np.arange(0,T+dt,dt)
    u0 = 0
    xx = np.array(x0)
    uu = np.array([0])
    ei = np.array([0,0,0,0])

    def du(in_,t,ep):
        return ep

    def ctrl(self,x):
        ep = x-self.xd
        self.ei = odeint(Model.du,self.ei,self.tt,args=(ep,))[-1]
        u = self.kp @ ep + self.ki @ self.ei
        return u

    def rp(in_,t,u):
        x = in_
        
        th1 = x[0]
        th2 = x[1]
        dth1 = x[2]
        dth2 = x[3]
        w = np.array([dth1,dth2])
        
        tau = np.array([u,0])
        m1 = 1
        m2 = 1
        L1 = 1
        L2 = 1
        g = 9.8
        f = np.array([0.6,0.6])

        D = np.array([[(m1+m2)*L1*L1+m2*L2*L2+2*m2*L1*L2*math.cos(th2), m2*L2*L2+m2*L1*L2*math.cos(th2)],
                     [m2*L2*L2+m2*L1*L2*math.cos(th2), m2*L2*L2]])

        C = np.array([-m2*L1*L2*math.sin(th2)*dth2*(2*dth1+dth2),
             m2*L1*L2*math.sin(th2)*dth1*dth1])

        g = np.array([(m1+m2)*g*L1*math.sin(th1)+m2*g*L2*math.sin(th1+th2),
            m2*g*L2*math.sin(th1+th2)])

        expr1 = tau - C - g - f*w
        ddth = np.linalg.inv(D) @ expr1
        dx = np.concatenate((w,
              ddth))
        
        return dx

    def state(self,u,x0):
        x = odeint(Model.rp,x0,self.tt,args=(u,))[-1,:]
        return x

    def step(self,x0,u0):
        u = self.ctrl(x0)
        x = self.state(u,x0)
        return (x,u)

    def main_cycle(self):
        x = self.x0
        u = self.u0
        for t in self.TT[1:]:
            (x,u) = self.step(x,u)
            self.xx = np.vstack((self.xx,x))
            self.uu = np.vstack((self.uu,u))

    def __init__(self):
        pass

m = Model()
m.main_cycle()
m.xx *= 180/math.pi

plt.figure(1)
plt.subplot(2,2,1)
plt.plot(m.TT,m.xx[:,0],label='th1')
plt.legend()
plt.grid()

plt.subplot(2,2,3)
plt.plot(m.TT,m.xx[:,1],label='th2')
plt.legend()
plt.grid()

plt.subplot(2,2,2)
plt.plot(m.TT,m.xx[:,2],label='dth1')
plt.legend()
plt.grid()

plt.subplot(2,2,4)
plt.plot(m.TT,m.xx[:,3],label='dth2')
plt.legend()
plt.grid()

plt.figure(2)
plt.plot(m.TT,m.uu,label='ctrl')
plt.legend()
plt.grid()

